# Terrascope gamma 0 download

In [ ]:
import openeo
import numpy as np
import geopandas as gpd
from pathlib import Path
import numpy as np
import xarray as xr
import hvplot.xarray
import pickle
import os
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
pad = Path(os.getcwd())
if pad.name != "Python":
    pad_correct = Path("../../Python")
    os.chdir(pad_correct)
from functions.pre_processing import pre_processing_pipeline

In [ ]:
shape_zwalm = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma.shp')
shape_zwalm.plot()
extent = shape_zwalm.total_bounds
print(extent)

In [ ]:
connection.list_collections()

In [ ]:
connection.describe_collection("TERRASCOPE_S1_GAMMA0_V1")

Deze dataset start op 07/06/2015

Bemerk dat ook 'TERRASCOPE_S2_LAI_V2' bevat vanaf 2015, lijkt met hogere resolutie te zijn (10m)

In [ ]:
collection = 'TERRASCOPE_S1_GAMMA0_V1' #Ground Range Detected
spatial_extent = {'west':extent[0],'east':extent[2],'south':extent[1],'north':extent[3]}
temporal_extent = ["2015-06-07", "2022-11-05"]  #start same as Hans, end where pywaterinfo forcings stop
bands = ['VV'] #enkel in deze geïnteresseerd 
# properties = {
#     "orbit_direction":lambda od: od == "ASCENDING"
# }
properties = {
    'sat:orbit_state': lambda od: od == "ASCENDING"
}

In [ ]:
list_temp_extent = []
list_temp_extent.append([temporal_extent[0],"2015-12-31"])
years = np.arange(2016,2023)
for year in np.arange(2016,2023):
    if year == 2022:
        #print([str(year)+"-01-01",temporal_extent[1]])
        list_temp_extent.append([str(year)+"-01-01",temporal_extent[1]])
    else:
        #print([str(year)+"-01-01",str(year)+ "-12-31"])
        list_temp_extent.append([str(year)+"-01-01",str(year)+ "-12-31"])
print(list_temp_extent)

In [ ]:
years = np.arange(2015,2023)
job_title_list = []
job_id_list = []
for i, temporal_extent in enumerate(list_temp_extent):
    s1a = connection.load_collection(
        collection_id = collection,
        spatial_extent= spatial_extent,
        temporal_extent = temporal_extent,
        #bands = bands,
        properties=properties
    )
    s1a = s1a.mask_polygon(shape_zwalm['geometry'].values[0])
    job_title = "s1a_terrascope-" +  str(years[i])
    job_title_list.append(job_title)
    job_s1a = s1a.create_job(title = job_title, out_format= 'NetCDF')
    job_s1a_id = job_s1a.job_id
    if job_s1a_id:
        print("Batch job created with id: ",job_s1a_id)
        job_s1a.start_job()
        job_id_list.append(job_s1a_id)
    else:
        print("Error! Job ID is None")

In [ ]:
if not os.path.exists('data/g0_a_terrascope'):
    os.makedirs('data/g0_a_terrascope')
for i,job_id in enumerate(job_id_list):
    job_connection = connection.job(job_id)
    results = job_connection.get_results()
    name_netcdf = job_title_list[i] + '.nc'
    filepath = "data/g0_a_terrascope/" + name_netcdf
    print(filepath)
    results.download_file(filepath)

## Visualisatie dataset

In [ ]:
sentinel_a_terrascope = xr.open_mfdataset('data/g0_a_terrascope/*.nc')
sentinel_a_terrascope

In [ ]:
sentinel_a_terrascope['VV'].hvplot.image('x','y',geo = True, crs = 3857, tiles = 'OSM')

Vragen voor Hans:
- Aanvaardbare preprocessing?
- resamplen kan, zowel op resolutie als op projectie! zo kan dus multilook naar grotere resolutie indien gewenst! zo van 10 x 10 m naar 20 x 20 m
- huidige projectie is https://epsg.io/32631  Dit is WGS84 maar dan in m precies

Bemerk: in OpenEO is dit beschrijven als een $\gamma_0$ product. Ik denk echter dat dit een $\sigma_0$. Dit is ook wat terrascope zelf zegt dat ze verschaffen, zie hiervoor [preprocessing_vito](https://terrascope.be/sites/terrascope/files/downloads/vito_s1-sigma0_grd.pdf)  

